In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame({
    'integer': [1, None, 1, 2, 1, 2, 3, 2],
    'float': [0.1, None, 0.1, 0.2, 0.1, 0.2, 0.3, 0.1],
    'categorical': ['a', 'b', 'a', 'b', 'a', None, 'c', None],
    'bool': [False, True, False, True, False, False, False, None],
    'nullable': [1, None, 3, None, 5, None, 7, None],
    'datetime': [
        '2010-01-01', '2010-02-01', '2010-01-01', '2010-02-01',
        '2010-01-01', '2010-02-01', '2010-03-01', None
    ]
})

In [2]:
data

,integer,float,categorical,bool,nullable,datetime
0,1.0,0.1,a,False,1.0,2010-01-01
1,NaN,NaN,b,True,NaN,2010-02-01
2,1.0,0.1,a,False,3.0,2010-01-01
3,2.0,0.2,b,True,NaN,2010-02-01
4,1.0,0.1,a,False,5.0,2010-01-01
5,2.0,0.2,None,False,NaN,2010-02-01
6,3.0,0.3,c,False,7.0,2010-03-01
7,2.0,0.1,None,None,NaN,None


In [3]:
data.to_csv('data.csv', index=False)

In [4]:
import json

metadata = {
    "path": "",
    "tables": [
        {
            "fields": [
                {
                    "name": "integer",
                    "type": "number",
                    "subtype": "integer",
                },
                {
                    "name": "float",
                    "type": "number",
                    "subtype": "float",
                },
                {
                    "name": "categorical",
                    "type": "categorical",
                    "subtype": "categorical",
                    "pii": False,
                    "pii_category": "email"
                },
                {
                    "name": "bool",
                    "type": "categorical",
                    "subtype": "bool",
                },
                {
                    "name": "nullable",
                    "type": "number",
                    "subtype": "float",
                },
                {
                    "name": "datetime",
                    "type": "datetime",
                    "format": "%Y-%m-%d"
                },
            ],
            "headers": True,
            "name": "data",
            "path": "data.csv",
            "use": True
        }
    ]
}

with open('data.meta.json', 'w') as f:
    json.dump(metadata, f, indent=4)

In [5]:
from sdv import SDV

sdv = SDV()
sdv.fit(metadata)

2019-10-08 15:04:18,669 - INFO - modeler - Modeling data
2019-10-08 15:04:18,670 - INFO - metadata - Loading table data
2019-10-08 15:04:18,715 - INFO - modeler - Modeling Complete


In [6]:
samples = sdv.sample_all()
samples['data']

,integer,float,categorical,bool,nullable,datetime
0,2.0,0.148526,NaN,NaN,NaN,NaT
1,3.0,0.169611,c,NaN,NaN,NaT
2,2.0,0.183588,b,True,NaN,2010-01-24 18:22:35.496082176
3,NaN,NaN,a,NaN,NaN,NaT
4,2.0,0.211784,b,True,6.055091,2010-02-03 07:10:02.057572352


In [7]:
from sdv.evaluation import evaluate

real = sdv.metadata.get_tables()

samples = sdv.sample_all(len(real['data']))

evaluate(real, samples).astype(str)

2019-10-08 15:04:24,046 - INFO - metadata - Loading table data
/home/xals/.virtualenvs/SDV/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


mse         0.06147794893540761
rmse        0.24794747212949683
r2_score     0.8621624152418914
dtype: object

In [8]:
from sdv import SDV

sdv = SDV()
sdv.fit('../tests/data/meta.json')

2019-10-08 15:04:27,649 - INFO - modeler - Modeling DEMO_ORDER_ITEMS
2019-10-08 15:04:27,650 - INFO - metadata - Loading table DEMO_ORDER_ITEMS
2019-10-08 15:04:27,659 - INFO - modeler - Modeling DEMO_ORDERS
2019-10-08 15:04:27,660 - INFO - metadata - Loading table DEMO_ORDERS
2019-10-08 15:04:27,667 - INFO - metadata - Loading table DEMO_ORDER_ITEMS
2019-10-08 15:04:27,670 - INFO - metadata - Loading table DEMO_ORDER_ITEMS
2019-10-08 15:04:27,756 - INFO - modeler - Modeling DEMO_CUSTOMERS
2019-10-08 15:04:27,756 - INFO - metadata - Loading table DEMO_CUSTOMERS
2019-10-08 15:04:27,766 - INFO - metadata - Loading table DEMO_ORDERS
2019-10-08 15:04:28,335 - INFO - modeler - Modeling Complete


In [16]:
from sdv.evaluation import evaluate

real = sdv.metadata.get_tables()

samples = sdv.sample_all(len(real['DEMO_CUSTOMERS']), reset_primary_keys=True)

evaluate(real, samples).astype(str)

2019-10-08 15:04:38,128 - INFO - metadata - Loading table DEMO_CUSTOMERS
2019-10-08 15:04:38,132 - INFO - metadata - Loading table DEMO_ORDERS
2019-10-08 15:04:38,136 - INFO - metadata - Loading table DEMO_ORDER_ITEMS
/home/xals/.virtualenvs/SDV/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


mse         1.0523891803778435e+50
rmse        1.0258602148333094e+25
r2_score        0.9999999999963233
dtype: object

In [17]:
samples['DEMO_CUSTOMERS'].describe()

,CUSTOMER_ID,PHONE_NUMBER1,CREDIT_LIMIT
count,7.000000,7.000000e+00,7.000000
mean,3.000000,7.356161e+09,1110.000000
std,2.160247,1.267037e+09,331.907618
min,0.000000,5.206541e+09,511.000000
25%,1.500000,6.559874e+09,1013.500000
50%,3.000000,7.892013e+09,1113.000000
75%,4.500000,8.356601e+09,1265.500000
max,6.000000,8.561623e+09,1588.000000


In [18]:
real['DEMO_CUSTOMERS'].describe()

,CUSTOMER_ID,PHONE_NUMBER1,CREDIT_LIMIT
count,7.000000e+00,7.000000e+00,7.000000
mean,2.219856e+07,6.464124e+09,1071.428571
std,3.907774e+07,1.374656e+09,449.867705
min,4.000000e+00,4.045553e+09,500.000000
25%,2.990810e+05,6.175553e+09,1000.000000
50%,6.304070e+05,6.175553e+09,1000.000000
75%,2.841125e+07,7.035552e+09,1000.000000
max,9.733881e+07,8.605552e+09,2000.000000


In [19]:
samples['DEMO_ORDERS'].describe()

,ORDER_ID,CUSTOMER_ID,ORDER_TOTAL
count,19.000000,19.000000,19.000000
mean,9.000000,2.947368,1774.894737
std,5.627314,1.899523,1530.278584
min,0.000000,0.000000,41.000000
25%,4.500000,1.500000,984.500000
50%,9.000000,3.000000,1285.000000
75%,13.500000,4.500000,2146.500000
max,18.000000,6.000000,6539.000000


In [20]:
real['DEMO_ORDERS'].describe()

,ORDER_ID,CUSTOMER_ID,ORDER_TOTAL
count,10.00000,1.000000e+01,10.000000
mean,5.50000,4.040089e+07,1397.500000
std,3.02765,4.518278e+07,722.322835
min,1.00000,4.000000e+00,730.000000
25%,3.25000,5.000000e+01,733.250000
50%,5.50000,2.799810e+07,1223.000000
75%,7.75000,8.700314e+07,2125.000000
max,10.00000,9.733881e+07,2380.000000


In [21]:
samples['DEMO_ORDER_ITEMS']

,ORDER_ITEM_ID,ORDER_ID,PRODUCT_ID,UNIT_PRICE,QUANTITY
0,0,0.0,5.0,54.0,3.0
1,1,0.0,3.0,66.0,5.0
2,2,0.0,4.0,58.0,4.0
3,3,0.0,5.0,54.0,1.0
4,4,0.0,3.0,73.0,5.0
5,5,1.0,7.0,105.0,5.0
6,6,1.0,10.0,39.0,2.0
7,7,1.0,16.0,28.0,0.0
8,8,1.0,2.0,119.0,4.0
9,9,1.0,9.0,61.0,1.0


In [22]:
real['DEMO_ORDER_ITEMS']

,ORDER_ITEM_ID,ORDER_ID,PRODUCT_ID,UNIT_PRICE,QUANTITY
0,100,10,7,52,8
1,101,8,6,125,4
2,102,1,6,125,4
3,103,4,9,125,4
4,104,1,9,113,4
5,105,9,10,87,2
6,106,10,6,39,4
7,107,1,6,50,4
8,108,2,3,31,2
9,109,4,6,37,3
